In [1]:
import sqlite3

print(sqlite3.version)
print(sqlite3.sqlite_version)

2.6.0
3.39.3


### Connect to the database. In memory in this case

In [2]:
dbconn = sqlite3.connect('mydw2') #permanent database

In [4]:
print(dbconn)

### Create a cursor to use to execute SQL statements.

In [5]:
cursor = dbconn.cursor()

In [6]:
cursor

### Use the cursor to execute SQL statements to the database
We always execute a commit after the statement.

In [7]:
cursor.execute('''CREATE TABLE IF NOT EXISTS books(id INTEGER PRIMARY KEY, 
                   title TEXT, author TEXT, price TEXT, year TEXT)
''')
dbconn.commit()

In [8]:
cursor.execute('''INSERT INTO books values (1, 'Pro PowerShell', 'Bryan Cafferky', 35.00, 2015)
''')
cursor.execute('''INSERT INTO books values (2, "'Hithiker's Guide to the Galaxy", 'Douglas Adams', 12.00, 199)
''')
dbconn.commit()

In [9]:
lstbooks = cursor.execute('''select * from books;''').fetchall()
print(lstbooks)

[(1, 'Pro PowerShell', 'Bryan Cafferky', '35.0', '2015'), (2, "'Hithiker's Guide to the Galaxy", 'Douglas Adams', '12.0', '199')]


In [10]:
type(lstbooks)

list

### Dataframes are much easier to work with so let's use a dataframe instead.

Dataframes are much easier to work with so let's use a dataframe instead.

In [12]:
import sqlite3
import pandas as pd

# convert query results to dataframe
dfbook = pd.read_sql_query("SELECT * FROM books", dbconn)
dfbook.head()

,id,title,author,price,year
0,1,Pro PowerShell,Bryan Cafferky,35.0,2015
1,2,'Hithiker's Guide to the Galaxy,Douglas Adams,12.0,199


In [13]:
type(dfbook)

pandas.core.frame.DataFrame

### We can drop the table since we are done.

In [14]:
cursor = dbconn.cursor()
cursor.execute('''DROP TABLE books''')
dbconn.commit()

In [15]:
booksdf = cursor.execute('''select * from books;''').fetchall()

OperationalError: no such table: books

In [16]:
ls

 Volume in drive D is Transcend
 Volume Serial Number is 1638-EF94

 Directory of D:\myPython\Bigdata

11/08/2022  07:32 PM    <DIR>          .
11/08/2022  07:32 PM    <DIR>          ..
04/15/2022  08:20 PM                47 .gitignore
11/08/2022  06:11 PM    <DIR>          .ipynb_checkpoints
11/08/2022  12:17 AM             5,264 20221107_Loading data from SQL Server to Python pandas dataframe.ipynb
11/08/2022  07:32 PM             8,843 20221108_Using SQLite for Data Analysis.ipynb
07/22/2022  05:20 AM    <DIR>          AdvancedQueryStore
07/22/2022  05:20 AM    <DIR>          AnalysisServices
07/22/2022  05:20 AM    <DIR>          AnalysisServicesModelTypes
07/22/2022  05:20 AM    <DIR>          ApacheAirflow
07/22/2022  05:20 AM    <DIR>          ApacheSparkSQL
05/15/2022  02:04 PM     2,088,635,836 Azure Databricks.mp4
07/22/2022  05:21 AM    <DIR>          AzureADFv2
07/22/2022  05:21 AM    <DIR>          AzureDatabricks_Overview
07/22/2022  05:21 AM    <DIR>          AzureDatabr

### Save a dataframe to SQLite

In [20]:
custDF = pd.read_csv('DimCustomer.csv')
custDF.head(2)

,CustomerKey,GeographyKey,CustomerAlternateKey,Title,FirstName,MiddleName,LastName,NameStyle,BirthDate,MaritalStatus,...,EnglishOccupation,SpanishOccupation,FrenchOccupation,HouseOwnerFlag,NumberCarsOwned,AddressLine1,AddressLine2,Phone,DateFirstPurchase,CommuteDistance
0,11000,26,AW00011000,NaN,Jon,V,Yang,0,1971-10-06,M,...,Professional,Profesional,Cadre,1,0,3761 N. 14th St,NaN,1 (11) 500 555-0162,2011-01-19,1-2 Miles
1,11001,37,AW00011001,NaN,Eugene,L,Huang,0,1976-05-10,S,...,Professional,Profesional,Cadre,0,1,2243 W St.,NaN,1 (11) 500 555-0110,2011-01-15,0-1 Miles


In [21]:
custDF.columns 

Index(['CustomerKey', 'GeographyKey', 'CustomerAlternateKey', 'Title',
       'FirstName', 'MiddleName', 'LastName', 'NameStyle', 'BirthDate',
       'MaritalStatus', 'Suffix', 'Gender', 'EmailAddress', 'YearlyIncome',
       'TotalChildren', 'NumberChildrenAtHome', 'EnglishEducation',
       'SpanishEducation', 'FrenchEducation', 'EnglishOccupation',
       'SpanishOccupation', 'FrenchOccupation', 'HouseOwnerFlag',
       'NumberCarsOwned', 'AddressLine1', 'AddressLine2', 'Phone',
       'DateFirstPurchase', 'CommuteDistance'],
      dtype='object')

In [22]:
custDF.dtypes.head(3)

CustomerKey              int64
GeographyKey             int64
CustomerAlternateKey    object
dtype: object

In [23]:
# if_exists{‘fail’, ‘replace’, ‘append’}, default ‘fail’

custDF.to_sql('DimCustomer', con=dbconn, if_exists='replace')

18484

In [24]:
pd.read_sql_query('select * from DimCustomer limit 2',con=dbconn)

,index,CustomerKey,GeographyKey,CustomerAlternateKey,Title,FirstName,MiddleName,LastName,NameStyle,BirthDate,...,EnglishOccupation,SpanishOccupation,FrenchOccupation,HouseOwnerFlag,NumberCarsOwned,AddressLine1,AddressLine2,Phone,DateFirstPurchase,CommuteDistance
0,0,11000,26,AW00011000,None,Jon,V,Yang,0,1971-10-06,...,Professional,Profesional,Cadre,1,0,3761 N. 14th St,None,1 (11) 500 555-0162,2011-01-19,1-2 Miles
1,1,11001,37,AW00011001,None,Eugene,L,Huang,0,1976-05-10,...,Professional,Profesional,Cadre,0,1,2243 W St.,None,1 (11) 500 555-0110,2011-01-15,0-1 Miles


In [25]:
cursor.close()
dbconn.close()

Chinook, SQLite's Answer to AdventureWorks...

In [28]:
conn = sqlite3.connect("chinook.db")
cur = conn.cursor()

albums = cur.execute("""select * from albums order by title limit 3;""").fetchall()
print(albums)

[(156, '...And Justice For All', 50), (257, '20th Century Masters - The Millennium Collection: The Best of Scorpions', 179), (296, 'A Copland Celebration, Vol. I', 230)]


In [29]:
type(albums)

list

In [30]:
cur.execute("""select * from albums order by title limit 3;""").fetchone()

(156, '...And Justice For All', 50)

In [31]:
albumdata = cur.execute("""select * from albums order by title limit 3;""").fetchmany(5)

In [32]:
# Convert the list returned to a dataframe..

from pandas import DataFrame

DataFrame(albumdata)

,0,1,2
0,156,...And Justice For All,50
1,257,20th Century Masters - The Millennium Collecti...,179
2,296,"A Copland Celebration, Vol. I",230


In [33]:
import sqlite3
import pandas as pd

# return query results as a dataframe

dfalbum = pd.read_sql_query("SELECT * FROM albums", conn)
dfalbum.head()

,AlbumId,Title,ArtistId
0,1,For Those About To Rock We Salute You,1
1,2,Balls to the Wall,2
2,3,Restless and Wild,2
3,4,Let There Be Rock,1
4,5,Big Ones,3


### SQLite has a database catalog

In [34]:
pd.read_sql_query("select * from sqlite_master where type = 'table';", conn)

,type,name,tbl_name,rootpage,sql
0,table,albums,albums,2,"CREATE TABLE ""albums""\r\n(\r\n [AlbumId] IN..."
1,table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
2,table,artists,artists,4,"CREATE TABLE ""artists""\r\n(\r\n [ArtistId] ..."
3,table,customers,customers,5,"CREATE TABLE ""customers""\r\n(\r\n [Customer..."
4,table,employees,employees,8,"CREATE TABLE ""employees""\r\n(\r\n [Employee..."
5,table,genres,genres,10,"CREATE TABLE ""genres""\r\n(\r\n [GenreId] IN..."
6,table,invoices,invoices,11,"CREATE TABLE ""invoices""\r\n(\r\n [InvoiceId..."
7,table,invoice_items,invoice_items,13,"CREATE TABLE ""invoice_items""\r\n(\r\n [Invo..."
8,table,media_types,media_types,15,"CREATE TABLE ""media_types""\r\n(\r\n [MediaT..."
9,table,playlists,playlists,16,"CREATE TABLE ""playlists""\r\n(\r\n [Playlist..."


In [35]:
pd.read_sql_query("select name from sqlite_master where type = 'index';", conn)

,name
0,sqlite_autoindex_playlist_track_1
1,IFK_AlbumArtistId
2,IFK_CustomerSupportRepId
3,IFK_EmployeeReportsTo
4,IFK_InvoiceCustomerId
5,IFK_InvoiceLineInvoiceId
6,IFK_InvoiceLineTrackId
7,IFK_PlaylistTrackTrackId
8,IFK_TrackAlbumId
9,IFK_TrackGenreId


In [36]:
dftablist = pd.read_sql_query("select name from sqlite_master where type = 'table';", conn)

In [37]:
# Get table schema...

pd.read_sql_query("""
PRAGMA table_info('albums');
""", conn)

,cid,name,type,notnull,dflt_value,pk
0,0,AlbumId,INTEGER,1,None,1
1,1,Title,NVARCHAR(160),1,None,0
2,2,ArtistId,INTEGER,1,None,0


### Suppressing the default index...

In [38]:
qry = '''
select substr(a.name, 1, 10) as Artist,
       substr(title,1, 15) as Album,
       t.name as Song
from albums r
join artists a
on (r.artistid = a.artistid)
join tracks t 
on (r.albumid = t.albumid)
order by a.name, title
limit 15;
'''

pd.read_sql_query(qry, conn, index_col = 'Artist')  # index_col replaces the default dataframe index

,Album,Song
Artist,,
AC/DC,For Those About,For Those About To Rock (We Salute You)
AC/DC,For Those About,Put The Finger On You
AC/DC,For Those About,Let's Get It Up
AC/DC,For Those About,Inject The Venom
AC/DC,For Those About,Snowballed
AC/DC,For Those About,Evil Walks
AC/DC,For Those About,C.O.D.
AC/DC,For Those About,Breaking The Rules
AC/DC,For Those About,Night Of The Long Knives


### Let's create a new table from a query

In [39]:
# Note the create table statment before the select query...

qry = '''
create table if not exists top15 as
select substr(a.name, 1, 10) as Artist,
       substr(title,1, 15) as Album,
       t.name as Song
from albums r
join artists a
on (r.artistid = a.artistid)
join tracks t 
on (r.albumid = t.albumid)
order by a.name, title
limit 15;
'''

albums = cur.execute(qry)

In [40]:
pd.read_sql_query('select * from top15', conn)

,Artist,Album,Song
0,AC/DC,For Those About,For Those About To Rock (We Salute You)
1,AC/DC,For Those About,Put The Finger On You
2,AC/DC,For Those About,Let's Get It Up
3,AC/DC,For Those About,Inject The Venom
4,AC/DC,For Those About,Snowballed
5,AC/DC,For Those About,Evil Walks
6,AC/DC,For Those About,C.O.D.
7,AC/DC,For Those About,Breaking The Rules
8,AC/DC,For Those About,Night Of The Long Knives
9,AC/DC,For Those About,Spellbound


Let's create a new table from a dataframe

In [41]:
best_acdcDF = pd.read_sql_query('select * from top15 where artist = "AC/DC"', conn)

### The works with full scale DBMSs like PostgreSQL too

In [43]:
best_acdcDF.to_sql('best_acdc', con=conn, if_exists='append')

15

In [44]:
pd.read_sql_query('select * from best_acdc', conn)

,index,Artist,Album,Song
0,0,AC/DC,For Those About,For Those About To Rock (We Salute You)
1,1,AC/DC,For Those About,Put The Finger On You
2,2,AC/DC,For Those About,Let's Get It Up
3,3,AC/DC,For Those About,Inject The Venom
4,4,AC/DC,For Those About,Snowballed
5,5,AC/DC,For Those About,Evil Walks
6,6,AC/DC,For Those About,C.O.D.
7,7,AC/DC,For Those About,Breaking The Rules
8,8,AC/DC,For Those About,Night Of The Long Knives
9,9,AC/DC,For Those About,Spellbound


close the connection

In [45]:
cur.close()
conn.close()

### Connecting to PostgreSQL

In [46]:
pip install -U psycopg2

Note: you may need to restart the kernel to use updated packages.


In [50]:
import psycopg2 as pg
import pandas.io.sql as psql
 
# get connected to the database
connection = pg.connect("dbname=movies user=postgres password=2Xjioljl2agiiodl")
 
df = pd.read_sql_query('select * from actors',con=connection)
df

C:\Users\user\anaconda3\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,actor_id,first_name,last_name,gender,date_of_birth,add_date,update_date
0,1,Malin,Akerman,F,1978-05-12,None,None
1,2,Tim,Allen,M,1953-06-13,None,None
2,3,Julie,Andrews,F,1935-10-01,None,None
3,4,Ivana,Baquero,F,1994-06-11,None,None
4,5,Lorraine,Bracco,F,1954-10-02,None,None
...,...,...,...,...,...,...,...
142,143,Ji-tae,Yoo,M,1976-04-13,None,None
143,144,Jin-seo,Yoon,F,1983-08-05,None,None
144,145,Sean,Young,F,1959-11-20,None,None
145,146,Billy,Zane,M,1966-02-24,None,None
